# HashiCorp Vault Demo with AWS IAM Roles Anywhere 

AWS has a feature called AWS IAM Roles Anywhere for workloads running outside of AWS.

This feature allow workloads running outside of AWS to assume AWS Credentials to perform actions (e.g. store backups on S3) and not have to use hardcoded AWS credentials.

This demo shows combine AWS IAM Roles Anywhere with HashiCorp Vault to use Vault as the Trust anchor to request AWS Credentials based on a PKI that is managed via Vault.

Ref: 
- https://developer.hashicorp.com/vault/docs/secrets/pki
- https://koudingspawn.de/combine-vault-with-iam-anywhere/

<img src="images/vault-demo-aws-iam-roles-anywhere.png">

## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup. The components used in this demo are:

- Vault Enterprise installed on docker (to simulate an external Vault)
- You have the Vault CLI installed

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200 and you have set your VAULT_ADDR and VAULT_TOKEN variables.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

In [ ]:
# Install AWS CLI
brew install awscli

In [ ]:
# Download AWS signing helper. This is used to obtain temporary security credentials from AWS IAM Roles Anywhere.
# Ref: https://docs.aws.amazon.com/rolesanywhere/latest/userguide/credential-helper.html
curl -o aws_signing_helper https://rolesanywhere.amazonaws.com/releases/1.1.1/X86_64/Darwin/aws_signing_helper

# Make the helper executable
chmod +x aws_signing_helper


In [ ]:
# Setup your AWS IAM programmatic keys that has permissions to configure AWS IAM Roles Anywhere.
# We will be using these credentials with the AWS CLI commands later.
# For demo purposes, you can create a user and attach the "AdministratorAccess" policy to it.
# Create an access key under this user and export the AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY values here.
# Ref: https://docs.aws.amazon.com/IAM/latest/UserGuide/security-creds-programmatic-access.html
export AWS_ACCESS_KEY_ID=
export AWS_SECRET_ACCESS_KEY=
unset AWS_SESSION_TOKEN

In [ ]:
# Set to your local region
aws configure set region ap-southeast-1

In [ ]:
# Verify that the credentials work. You should see your configured IAM users in your AWS account.
aws iam list-users

## Setup Vault as a CA to serve as the Trust Anchor

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

In [ ]:
# Enable PKI engine at the default mount path
vault secrets enable pki

In [ ]:
# configure max lease ttl on certificates - 8760h or 365 days
vault secrets tune -max-lease-ttl=8760h pki

In [ ]:
# Create the CA, the private key is kept inside Vault.
# Alternatively, generate an intermediate CA and sign with your root CA.
# CA cert is valid for 8760h or 365 days
vault write pki/root/generate/internal \
    common_name=my-org.com \
    ttl=8760h

In [ ]:
# configure Vault with the URL's for CRL
vault write pki/config/urls \
    issuing_certificates="http://127.0.0.1:8200/v1/pki/ca" \
    crl_distribution_points="http://127.0.0.1:8200/v1/pki/crl"

In [ ]:
# Store the CA certificate, we will use this later when setting up the trust anchor in AWS IAM Roles Anywhere.
curl http://127.0.0.1:8200/v1/pki/ca_chain > ca.crt
echo
echo "CA Certificate:"
cat ca.crt


## Configure AWS IAM Roles Anywhere

1. Setup Trust Anchor
2. Configure IAM Role
3. Configure IAM Policy
4. Attach IAM Policy to IAM Role
5. Create IAM Roles Anywhere Profile with IAM Role details

In [ ]:
# Create a trust anchor in AWS IAM Roles Anywhere using Vault's CA certificate.
aws rolesanywhere create-trust-anchor \
    --name "VaultTrustAnchor" \
    --source "sourceType=CERTIFICATE_BUNDLE,sourceData={x509CertificateData=$(cat ca.crt)}" \
    --output json > trustanchor.json
echo
cat trustanchor.json

# Enable the trust anchor
export TRUST_ANCHOR_ID=$(jq -r .trustAnchor.trustAnchorId < trustanchor.json)
aws rolesanywhere enable-trust-anchor --trust-anchor-id $TRUST_ANCHOR_ID

# Store the Trust Anchor ARN to be used by the aws_signing_helper
export TRUST_ANCHOR_ARN=$(jq -r .trustAnchor.trustAnchorArn < trustanchor.json)


In [ ]:
# Verify that the trust anchor was created
aws rolesanywhere list-trust-anchors --no-paginate

In [ ]:
# Create an IAM role that can be assumed by the on-prem machine
aws iam create-role --role-name on-prem-instance-role --assume-role-policy-document file://assume-role-doc.json --output json > role.json
echo
cat role.json

# Store the Role ARN to be used when creating the IAM Roles Anywhere Profile and by the aws_signing_helper 
export ROLE_ARN=$(jq -r .Role.Arn < role.json)

In [ ]:
# Create a policy for the permissions used by the IAM role
cat on-prem-policy-doc.json
echo
aws iam create-policy \
    --policy-name on-prem-instance-policy \
    --policy-document file://on-prem-policy-doc.json --output json > policy.json
echo
cat policy.json

# Store the Policy ARN to be used when creating the IAM Roles Anywhere Profile
export POLICY_ARN=$(jq -r .Policy.Arn < policy.json)

In [ ]:
# Attach the policy to the role
aws iam attach-role-policy --role-name on-prem-instance-role --policy-arn $POLICY_ARN

# List the policies attached to the role
aws iam list-attached-role-policies --role-name on-prem-instance-role

In [ ]:
# Create an IAM Roles Anywhere profile that links the earlier created IAM role and policy.
echo "Role ARN: $ROLE_ARN"
#echo "Policy ARN: $POLICY_ARN"

aws rolesanywhere create-profile \
    --name "OnPremInstanceProfile" \
    --role-arns $ROLE_ARN \
    --output json > profile.json
# Adding this did not work. Added the policy to the role instead.
#    --managed-policy-arns $POLICY_ARN \

echo
cat profile.json

# Enable the profile
export PROFILE_ID=$(jq -r .profile.profileId < profile.json)
aws rolesanywhere enable-profile --profile-id $PROFILE_ID

# Store the Profile ARN to be used by the aws_signing_helper
export PROFILE_ARN=$(jq -r .profile.profileArn < profile.json)

## Create certificate for on-prem instance and use aws_signing_helper to get temporary AWS credentials

In [ ]:
# Configure certificate for the on-prem instance.  Set TTL to 30s for demo.
vault write pki/roles/on-prem-instance \
    allowed_domains=on-prem-instance \
    allow_bare_domains=true \
    allow_subdomains=true \
    max_ttl=30s

In [ ]:
# Generate the cert for use by IAM Roles Anywhere
# Note the TTL is set to 30s for demo purposes.
vault write -format=json pki/issue/on-prem-instance \
    common_name="on-prem-instance" > cert.json
jq -r .data.certificate < cert.json > cert.pem
jq -r .data.private_key < cert.json > private.pem

In [ ]:
# Request temporary AWS IAM Credentials via AWS IAM Roles Anywhere
# If you run this command 30 seconds later, you will see an "AccessDeniedException: Untrusted signing certificate" error.
# This is expected due to the configured TTL.
echo "Trust Anchor ARN: $TRUST_ANCHOR_ARN"
echo "Profile ARN: $PROFILE_ARN"
echo "Role ARN: $ROLE_ARN"

./aws_signing_helper credential-process \
    --certificate cert.pem \
    --private-key private.pem \
    --trust-anchor-arn $TRUST_ANCHOR_ARN \
    --profile-arn $PROFILE_ARN \
    --role-arn $ROLE_ARN > aws_credentials.json
echo
cat aws_credentials.json | jq

In [ ]:
# Verify that the returned temporary credentials work. You should see your configured IAM users in your AWS account.
AWS_ACCESS_KEY_ID=$(jq -r .AccessKeyId < aws_credentials.json) \
AWS_SECRET_ACCESS_KEY=$(jq -r .SecretAccessKey < aws_credentials.json) \
AWS_SESSION_TOKEN=$(jq -r .SessionToken < aws_credentials.json) \
aws iam list-users

# Cleanup

In [ ]:
# Cleanup

# Delete the trust anchor
export trustanchorid=$(jq -r .trustAnchor.trustAnchorId < trustanchor.json)
echo "Deleting Anchor ID: $trustanchorid"
aws rolesanywhere delete-trust-anchor --trust-anchor-id $trustanchorid

# Delete the profile
aws rolesanywhere delete-profile --profile-id $PROFILE_ID

# Detach the policy from the role
aws iam detach-role-policy --role-name on-prem-instance-role --policy-arn $POLICY_ARN

# Delete IAM role
aws iam delete-role --role-name on-prem-instance-role

# Delete IAM policy
aws iam delete-policy --policy-arn $(jq -r .Policy.Arn < policy.json)

# Make sure this portion is cleaned up before you clean up the temp files.

In [ ]:
# Remove temp files
rm aws_credentials.json
rm ca.crt
rm trustanchor.json
rm role.json
rm policy.json
rm profile.json
rm cert.json
rm cert.pem
rm private.pem

# Disable secrets engine
vault secrets disable pki

# Stop Vault
docker stop vault-enterprise

In [ ]:
# Remove the aws_signing_helper (Optional). You can skip this if you plan to re-run the demo.
rm aws_signing_helper

# Other Useful Commands